## Config

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime, timedelta

In [2]:
from nixtlats import TimeGPT
token = 'REDACTED'

timegpt = TimeGPT(token = token)

timegpt.validate_token()

INFO:nixtlats.timegpt:Happy Forecasting! :), If you have questions or need support, please email ops@nixtla.io


True

## Read data

In [3]:
df = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/M3 Data/M3_quarter_processed.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37004 entries, 0 to 37003
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Series       37004 non-null  int64         
 1   N            37004 non-null  int64         
 2   NF           37004 non-null  int64         
 3   Category     37004 non-null  object        
 4   Measurement  37004 non-null  int64         
 5   Value        37004 non-null  float64       
 6   Date         37004 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 2.0+ MB


## Function to perform predictions

In [10]:
def simple_forecast_with_timegpt(df):
    forecasts_list = []  # List to hold forecast dataframes for each series
    call_count = 0  # Initialize call count
    start_time = datetime.now()  # Track the start time
    
    for series_id in df['Series'].unique():
        current_time = datetime.now()
        
        # Check if we are approaching the rate limit
        if call_count >= 200:
            # Calculate remaining time to the next minute
            time_to_next_minute = 60 - current_time.second + (1000000 - current_time.microsecond) / 1000000.0
            print(f"Pausing for {time_to_next_minute:.2f} seconds to comply with rate limit.")
            time.sleep(time_to_next_minute)  # Pause execution
            
            # Reset the call count and start time for the new minute
            call_count = 0
            start_time = datetime.now()
        
        series_df = df[df['Series'] == series_id].sort_values('Date')  # Filter and sort by date
        series_df.dropna(subset=['Value'], inplace=True)  # Drop missing values
        
        n = int(series_df['N'].iloc[0])
        nf = int(series_df['NF'].iloc[0])  
        
        # Ensure the dataframe passed does not use more than (N - NF) examples
        trimmed_df = series_df.head(n - nf)
        
        try:
            forecast_df = timegpt.forecast(trimmed_df, h=nf, id_col='Series', time_col='Date', target_col='Value')
            forecasts_list.append(forecast_df)  
        # Call TimeGPT model with nf steps forecast
        except Exception as e:
            print(f"Error encountered for series {series_id}: {e}")
        
        call_count += 1  # Increment call count after each API call
        
        # Check if a minute has passed; if so, reset the counter
        if (datetime.now() - start_time).seconds >= 60:
            call_count = 0
            start_time = datetime.now()

    # Concatenate all forecast DataFrames
    final_forecasts_df = pd.concat(forecasts_list, ignore_index=True)
    
    return final_forecasts_df

## Run the function

In [11]:
all_forecasts = simple_forecast_with_timegpt(df)

INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: QS-OCT
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: QS-OCT
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: QS-OCT
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: QS-OCT
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: QS-OCT
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.tim

Error encountered for series 1095: status_code: 500, body: {'data': None, 'message': 'Request failed with status code 500', 'code': 'B30', 'requestID': '5VGWUWPPSG', 'support': 'If you have questions or need support, please email ops@nixtla.io'}


INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: QS-OCT
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: QS-OCT
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: QS-OCT
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: QS-OCT
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: QS-OCT
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixt

Error encountered for series 1095: status_code: 500, body: {'data': None, 'message': 'Request failed with status code 500', 'code': 'B30', 'requestID': '5VGWUWPPSG', 'support': 'If you have questions or need support, please email ops@nixtla.io'}

In [12]:
all_forecasts

,Series,Date,TimeGPT
0,646,1993-01-01,5437.876953
1,646,1993-04-01,5440.134766
2,646,1993-07-01,5454.375000
3,646,1993-10-01,5564.521973
4,646,1994-01-01,5614.047852
...,...,...,...
6035,1401,1974-10-01,4579.938477
6036,1401,1975-01-01,4569.582031
6037,1401,1975-04-01,4653.845703
6038,1401,1975-07-01,4854.271484


In [15]:
print(len(all_forecasts['Series'].unique()))


755


In [16]:
all_forecasts.loc[all_forecasts['Series'] == 1095]

,Series,Date,TimeGPT


### Do seperate forecast for error

In [17]:
failed = df.loc[df['Series'] == 1095]
failed

,Series,N,NF,Category,Measurement,Value,Date
22128,1095,52,8,MACRO,1,2536.70,1980-01-01
22129,1095,52,8,MACRO,2,2758.60,1980-04-01
22130,1095,52,8,MACRO,3,2659.22,1980-07-01
22131,1095,52,8,MACRO,4,2958.10,1980-10-01
22132,1095,52,8,MACRO,5,2772.08,1981-01-01
22133,1095,52,8,MACRO,6,3005.70,1981-04-01
22134,1095,52,8,MACRO,7,2846.48,1981-07-01
22135,1095,52,8,MACRO,8,3155.96,1981-10-01
22136,1095,52,8,MACRO,9,2914.52,1982-01-01
22137,1095,52,8,MACRO,10,3150.16,1982-04-01


In [18]:
# trimmed_df.drop(columns = 'Category', inplace=True)
n = int(failed['N'].iloc[0])  # Assuming N is constant within each series
nf = int(failed['NF'].iloc[0])  # Assuming NF is constant within each series

failed = failed.head(n - nf)
print(n)
print(nf)
print(len(failed))

52
8
44


In [20]:
failed_forecast = timegpt.forecast(failed, h=nf, id_col='Series', time_col='Date', target_col='Value')
failed_forecast

INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: QS-OCT
INFO:nixtlats.timegpt:Calling Forecast Endpoint...


,Series,Date,TimeGPT
0,1095,1991-01-01,5113.579102
1,1095,1991-04-01,5179.778320
2,1095,1991-07-01,5101.454590
3,1095,1991-10-01,5188.261719
4,1095,1992-01-01,5099.959961
5,1095,1992-04-01,5137.158203
6,1095,1992-07-01,5085.696777
7,1095,1992-10-01,5194.062500


In [21]:
complete = pd.concat([all_forecasts, failed_forecast])
complete.sort_values(['Series', 'Date'], inplace=True)
complete

,Series,Date,TimeGPT
0,646,1993-01-01 00:00:00,5437.876953
1,646,1993-04-01 00:00:00,5440.134766
2,646,1993-07-01 00:00:00,5454.375000
3,646,1993-10-01 00:00:00,5564.521973
4,646,1994-01-01 00:00:00,5614.047852
...,...,...,...
6035,1401,1974-10-01 00:00:00,4579.938477
6036,1401,1975-01-01 00:00:00,4569.582031
6037,1401,1975-04-01 00:00:00,4653.845703
6038,1401,1975-07-01 00:00:00,4854.271484


### Save to parquet

In [22]:
complete['Date'] = pd.to_datetime(complete['Date'])
all_forecasts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Series   6040 non-null   int64         
 1   Date     6040 non-null   datetime64[ns]
 2   TimeGPT  6040 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 141.7 KB


In [23]:
complete.to_parquet('M3_quarter_simple_forecasts.parquet', compression='snappy', engine='pyarrow')